In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd

In [3]:
df=pd.DataFrame([[8,8,1],[7,9,1],[6,10,0],[5,5,0]],columns=['cgpa','profile_score','placement'])

In [4]:
df.head()

,cgpa,profile_score,placement
0,8,8,1
1,7,9,1
2,6,10,0
3,5,5,0


In [7]:
def initialize_parameters(layer_dim):
    
    np.random.seed(3)
    
    parameters={}
    L=len(layer_dim)
    
    for l in range(1,L):
        
        parameters['W' + str(l)]=np.ones((layer_dim[l-1],layer_dim[l]))*0.1
        parameters['b' + str(l)]=np.ones((layer_dim[l],1))
    
    return parameters

In [8]:
initialize_parameters([2,2,1])

{'W1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[1.],
        [1.]]),
 'W2': array([[0.1],
        [0.1]]),
 'b2': array([[1.]])}

In [9]:
#Utility_Function
def sigmoid(z):
    
    A= 1/(1+np.exp(-z))
    return A

In [10]:
def linear_forward(A_prev,w,b):
    
    z = np.dot(w.T,A_prev) + b
    
    A = sigmoid(z)
    
    return A

In [36]:
#Forward Prop
def L_layer_forward(X,parameters):
    
    A = X
    L=len(parameters) // 2  #Number of layers in the neural network
    
    for l in range(1,L+1):
        A_prev = A
        wl = parameters['W' + str(l)]
        bl = parameters['b' + str(l)]
        #print('A'+str(l-1)+': ',A_prev)
        #print('W'+str(l)+': ',wl)
        #print('b'+str(l)+': ',bl)
        #print('--'*20)
        
        A=linear_forward(A_prev,wl,bl)
        #print('A'+str(l)+': ',A)
        #print('**'*20)
    
    return A,A_prev

In [14]:
X=df[['cgpa','profile_score']].values[0].reshape(2,1) #Shape(no. of Features, no. of training example)
y=df[['placement']].values[0][0]
parameters = initialize_parameters([2,2,1])

y_hat,A1=L_layer_forward(X , parameters)

A0:  [[8]
 [8]]
W1:  [[0.1 0.1]
 [0.1 0.1]]
b1:  [[1.]
 [1.]]
----------------------------------------
A1:  [[0.93086158]
 [0.93086158]]
****************************************
A1:  [[0.93086158]
 [0.93086158]]
W2:  [[0.1]
 [0.1]]
b2:  [[1.]]
----------------------------------------
A2:  [[0.76605579]]
****************************************


In [15]:
y_hat=y_hat[0][0]

In [16]:
y_hat

0.7660557871456097

In [17]:
A1

array([[0.93086158],
       [0.93086158]])

In [51]:
def update_parameters(parameters,y,y_hat,A1,X):
    
    parameters['W2'][0][0] = parameters['W2'][0][0] + (0.001 * (y - y_hat) * A1[0][0])
    parameters['W2'][1][0] = parameters['W2'][1][0] + (0.001 * (y - y_hat)*A1[1][0])
    parameters['b2'][0][0] = parameters['b2'][0][0] + (0.001 * (y - y_hat))
    
    parameters['W1'][0][0] = parameters['W1'][0][0] + (0.001 * (y - y_hat) * parameters['W2'][0][0] * A1[0][0]*(1 - A1[0][0])* X[0][0])
    parameters['W1'][0][1] = parameters['W1'][0][1] + (0.001 * (y - y_hat) * parameters['W2'][0][0] * A1[0][0]*(1 - A1[0][0]) * X[1][0])
    parameters['b1'][0][0] = parameters['b1'][0][0] + (0.001 * (y - y_hat) * parameters['W2'][0][0] * A1[0][0]*(1 - A1[0][0]))
    
    parameters['W1'][1][0] = parameters['W1'][1][0] + (0.001 * (y - y_hat) * parameters['W2'][1][0] *A1[1][0]*(1 - A1[1][0])* X[0][0])
    parameters['W1'][1][1] = parameters['W1'][1][1] + (0.001 * (y - y_hat) * parameters['W2'][1][0]*A1[1][0]*(1 - A1[1][0]) * X[1][0])
    parameters['b1'][1][0] = parameters['b1'][1][0] + (0.001 * (y - y_hat) * parameters['W2'][1][0] * A1[1][0]*(1 - A1[1][0]))

In [19]:
update_parameters(parameters,y,y_hat,A1,X)

In [20]:
parameters

{'W1': array([[0.10001207, 0.10001207],
        [0.10001207, 0.10001207]]),
 'b1': array([[1.00000151],
        [1.00000151]]),
 'W2': array([[0.10021777],
        [0.10021777]]),
 'b2': array([[0.10045171]])}

In [34]:
def fit(X_train,y_train,epochs=5):
    
    parameters= initialize_parameters([2,2,1])
    
    for i in range(epochs):
        
        Loss=[]
        
        for j in range(X_train.shape[0]):
            
            X = X_train.iloc[j,:].values.reshape(2,1)
            y = y_train.iloc[j,-1:].values 
            
            #Parameter Initialization
            
            y_hat,A1= L_layer_forward(X,parameters)
            
            y_hat= y_hat[0][0]
            
            update_parameters(parameters,y,y_hat,A1,X)
            
            Loss.append(-y*np.log(y_hat) - (1-y)*np.log(1-y_hat))
        
        print('Epoch - ',i+1,'Loss - ',np.array(Loss).mean())
    
    print(parameters)

In [22]:
X_train = df.iloc[:,:-1]

In [26]:
y_train = df.iloc[:,-1:]

In [57]:
fit(X_train,y_train,epochs=1000)

Epoch -  1 Loss -  0.8577282208083334
Epoch -  2 Loss -  0.8569926844613232
Epoch -  3 Loss -  0.8562601105285556
Epoch -  4 Loss -  0.8555304897868045
Epoch -  5 Loss -  0.8548038130204954
Epoch -  6 Loss -  0.8540800710219603
Epoch -  7 Loss -  0.8533592545916855
Epoch -  8 Loss -  0.8526413545385596
Epoch -  9 Loss -  0.8519263616801143
Epoch -  10 Loss -  0.8512142668427642
Epoch -  11 Loss -  0.8505050608620406
Epoch -  12 Loss -  0.849798734582824
Epoch -  13 Loss -  0.8490952788595709
Epoch -  14 Loss -  0.848394684556538
Epoch -  15 Loss -  0.847696942548005
Epoch -  16 Loss -  0.8470020437184873
Epoch -  17 Loss -  0.8463099789629542
Epoch -  18 Loss -  0.8456207391870362
Epoch -  19 Loss -  0.8449343153072324
Epoch -  20 Loss -  0.844250698251114
Epoch -  21 Loss -  0.8435698789575239
Epoch -  22 Loss -  0.8428918483767734
Epoch -  23 Loss -  0.8422165974708347
Epoch -  24 Loss -  0.8415441172135318
Epoch -  25 Loss -  0.8408743985907261
Epoch -  26 Loss -  0.8402074326005006